# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [10]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_layer = 1

In [11]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=output_layer, activation="sigmoid"))

In [12]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 3s 9ms/step - loss: 0.7124 - accuracy: 0.4679
Epoch 2/50
38/38 [==============================] - 0s 8ms/step - loss: 0.6893 - accuracy: 0.5371
Epoch 3/50
38/38 [==============================] - 0s 10ms/step - loss: 0.6746 - accuracy: 0.5963
Epoch 4/50
38/38 [==============================] - 0s 5ms/step - loss: 0.6624 - accuracy: 0.6522
Epoch 5/50
38/38 [==============================] - 0s 7ms/step - loss: 0.6524 - accuracy: 0.6739
Epoch 6/50
38/38 [==============================] - 0s 6ms/step - loss: 0.6434 - accuracy: 0.6872
Epoch 7/50
38/38 [==============================] - 0s 8ms/step - loss: 0.6356 - accuracy: 0.6931
Epoch 8/50
38/38 [==============================] - 0s 7ms/step - loss: 0.6278 - accuracy: 0.6964
Epoch 9/50
38/38 [==============================] - 0s 5ms/step - loss: 0.6210 - accuracy: 0.7048
Epoch 10/50
38/38 [==============================] - 0s 7ms/step - loss: 0.6146 - accuracy: 0.7139
Epoch 11/50
38/38 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5420 - accuracy: 0.7325 - 195ms/epoch - 15ms/step
Loss: 0.5420169234275818, Accuracy: 0.7325000166893005


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
file_path = '/content/drive/My Drive'

# Save the model
nn_model.save(file_path + '/student_loans.keras')

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
# Set the model's file path
file_path = '/content/drive/My Drive/student_loans.keras'

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [18]:
# Make predictions with the test data
predictions = nn_model.predict(X_test_scaled,verbose=2)

# Display a sample of the predictions
predictions

13/13 - 0s - 111ms/epoch - 9ms/step


array([[0.5595816 ],
       [0.39456096],
       [0.75414854],
       [0.7050546 ],
       [0.75414854],
       [0.74366814],
       [0.75414854],
       [0.09792248],
       [0.70178103],
       [0.34716728],
       [0.75408095],
       [0.23094068],
       [0.44241205],
       [0.75414854],
       [0.5697344 ],
       [0.5243072 ],
       [0.75414854],
       [0.44672891],
       [0.6370443 ],
       [0.38851508],
       [0.6546787 ],
       [0.75414854],
       [0.32305646],
       [0.75414854],
       [0.2883881 ],
       [0.75414854],
       [0.75414854],
       [0.57517684],
       [0.41447988],
       [0.7437476 ],
       [0.6567833 ],
       [0.75414854],
       [0.28645808],
       [0.75414854],
       [0.2899543 ],
       [0.57999915],
       [0.23994598],
       [0.41154933],
       [0.75414854],
       [0.19121835],
       [0.75414854],
       [0.04689818],
       [0.08285965],
       [0.75414854],
       [0.07157957],
       [0.60240763],
       [0.14867853],
       [0.350

In [19]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [20]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.71      0.73      0.72       188
           1       0.75      0.74      0.74       212

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.73      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

The following data would need to be collected to build a recommendation system to recommend student loan options for students:
1. Degree Level - Undergrad, Junior College, Graduate or PhD. This information will indicate future income potential and repayment confidence level. Graduate and PhD students, more mature, are more serious with financial obligations.
2. Loan Requested (Annual) - Loan amount requested by the student for the academic year. This value will help narrow down student loan providers.
3. Current Income - This information will help access student's current financial position.
4. Current Financial Aid (per semester) - Any financial aid received by the student currently. Some providers will take this data into consideration.
5. Current Loan Total - Total loan amount ($). This will help determine total student loan providers are willing to authorize, irrespective of the loan amount being requested.  
6. Time To Graduation - This data will provide when the first loan repayment installment will be due to the provider.
7. College Major Degree - This information will provide future earning potential.
8. Credit Score (if available) - This score might not be available for many Undergrad and Junior College students but older students in higher degree level might have it. This value will provide credit worthiness of each student, as calculated by financial institutions.
9. Finance Workshop Attendance - This indicates if the student understands their repayment obligations and first amount due. Also showns their seriousness about loan/money matters. Attendance is pre-requisite for many student loan providers.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Based on the data chosen for the recommendation system, the model will use content-based filtering. The data collected from each student will be used by the content-based filtering process to generate unique recommendation for each student. As the chances of all features value being the same for different students are very low, content-based filtering for each individual student will be most appropriate.


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

Two real-worl challenges to take into consideration while building a recommendation system for student loans are:
1. Data Accuracy - The recommendation system can only be as accurate as the accuracy of the data entered by the student. The system does not have a process to validate the accuracy of data entered by each student using the system. If any feature value is incorrectly entered by the student (intentionally or unintentionally), the loan options recommended will not be as accurate.

2. Source Data Imbalance - If the source data is not balanced, the students with unique values for different features might not be provided with the most optimal loan recommendation. The train data needs to be diverse enough to account for many unique socio-economic status of students using the recommendation system for optimal loan recommendation.

